<a href="https://colab.research.google.com/github/louis103/GEE-Python-Tutorials/blob/main/Getting_Started_with_GEE_PythonAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Python gee library
import ee
# Trigger the GEE authetication flow
ee.Authenticate()
# Initialize the module throughout the Notebook.
ee.Initialize()

In [5]:
# Load a sample Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

In [6]:
# Get image properties
print(img.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 270285, 0, -30, 4414815]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 270285, 0, -30, 4414815]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 270285, 0, -30, 4414815]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 270285, 0, -30, 4414815]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7961, 7301], 'crs': 'EPSG:32613', 'crs_transform': [30, 0, 2702

# **Unsupervised** **Classification**

In [9]:
# Define a region of interest
region = ee.Geometry.Rectangle(31.56, -26.24, 31.78, -26.09)

# Load the landsat data
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01','2017-12-31')\
.filterBounds(region)\
.sort('CLOUD_COVER')\
.first()

# Make the training data
training_data = image.sample(**{
    'region' : region,
    'scale' : 30,
    'numPixels' : 5000
})

# Initiate the clusterer algorithm and train it
clusterer = ee.Clusterer.wekaKMeans(15).train(training_data)

# Cluster our image using the the unsupervised classification
result = image.cluster(clusterer)

In [10]:
# Import python leaflet mapping library (folium) and show\
# the clustered image on map
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
my_map = folium.Map(location=[-26.1694, 31.6632], zoom_start=11)

# Add the earth engine layer to map
my_map.add_ee_layer(result.clip(region).randomVisualizer(), {}, "Unsupervised Classification")

# Add layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)

# **Supervised Classification CART**

In [11]:
# CART supervised classification with landsat
# Region of Interest (ROI)
roi = ee.Geometry.Point([-122.26, 37.87])

# Load the landsat imagery
image1 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01','2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()

visParamsTrue = {'bands': ['B4','B3','B2'], 'min':0, 'max': 3000, 'gamma': 1.4}

In [26]:
# Get the Download URL for the training Feature Collection
ee.FeatureCollection('users/wambualouis/Train_Cover_CA').getDownloadURL()

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/99f870040e930cd29101a0feb19fee22-35058b9bb979134c0cf5999ba2348958:getFeatures'

In [27]:
# Import training data
training_fcoll = ee.FeatureCollection('users/wambualouis/Train_Cover_CA')

label = 'Class'
bands = ['B1','B2','B3','B4','B5','B7']
input = image1.select(bands)

# Overlay the points on the imagery to get training data
trainImage = input.sampleRegions(**{
    'collection': training_fcoll,
    'properties':[label],
    'scale':30,
})

trainingData = trainImage.randomColumn()
trainSet = trainingData.filter(ee.Filter.lessThan('random', 0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', 0.8))

In [28]:
# Classification Model
classifier = ee.Classifier.smileCart().train(trainSet, label, bands)

# Classify the image
classified = input.classify(classifier)

# Define a palette for the classification
landCoverPalette = [
    '#0c2c84', # Water(0)
    '#e31a1c', # Urban (1)
    '#005a32', # Forest (2)
    '#FF8000', # Crops (3)
    '#969696', # Barren Land (5)
]

In [29]:
# Import python leaflet mapping library (folium) and show\
# the clustered image on map
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
my_map = folium.Map(location=[38, -124], zoom_start=8)

# Add the earth engine layer to map
my_map.add_ee_layer(image1, visParamsTrue, "Landsat Image")
my_map.add_ee_layer(classified, {'palette': landCoverPalette, 'min':0,'max':4} , 'CART Classified Image')

# Add layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)

# Supervised Classification - Random Forest

In [32]:
# Supervised Classification using Random Forest
Rn_classifier = ee.Classifier.smileRandomForest(10).train(**{
    'features': trainSet,
    'classProperty':label,
    'inputProperties':bands,
})

# Classify the image
R_classified = input.classify(Rn_classifier)

In [33]:
# Import python leaflet mapping library (folium) and show\
# the clustered image on map
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
my_map = folium.Map(location=[38, -124], zoom_start=8)

# Add the earth engine layer to map
my_map.add_ee_layer(image1, visParamsTrue, "Landsat Image")
my_map.add_ee_layer(R_classified, {'palette': landCoverPalette, 'min':0,'max':4} , 'Random Forest Classified Image')

# Add layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)